### This Notebook obtains the Base Philosopher Data (Built upon later in the project) from http://www.philosophybasics.com/

In [1]:
from requests import get
from bs4 import BeautifulSoup
from unidecode import unidecode
from collections import defaultdict
import sys
import os
sys.path.append(os.path.abspath('../code'))
from default_ordered_dict import DefaultOrderedDict
import re
import string
import urllib
import os

In [157]:
r = get('http://www.philosophybasics.com/philosophers_anaximander.html')
r2 = get('http://www.philosophybasics.com/philosophers_thales.html')
soup = BeautifulSoup(r.content, 'html.parser')
soup2 = BeautifulSoup(r2.content, 'html.parser')

In [158]:
content = soup.select('p b')
content2 = soup2.select('p b')

In [159]:
content
content2

[<b>Thales of Miletus</b>,
 <b>Greek</b>,
 <b>Miletus</b>,
 <b>Ionia</b>,
 <b>Seven Sages of Greece</b>,
 <b>first philosopher</b>,
 <b>founder</b>,
 <b>teacher</b>,
 <b>substance</b>,
 <b>material objects</b>,
 <b>water</b>,
 <b>universality</b>,
 <b>"first scientist".</b>,
 <b>Miletus</b>,
 <b>Diogenes La\xebrtius</b>,
 <b>Examyas</b>,
 <b>Cleobulina</b>,
 <b>Thelidae</b>,
 <b>Agenor</b>,
 <b>Cadmus</b>,
 <b>Thebes</b>,
 <b>Phoenician</b>,
 <b>sketchy</b>,
 <b>contradictory</b>,
 <b>married</b>,
 <b>Cybisthus</b>,
 <b>Cybisthon</b>,
 <b>never married</b>,
 <b>no writings</b>,
 <b><i>"On the Solstice"</i></b>,
 <b><i>"On the Equinox"</i></b>,
 <b>business</b>,
 <b>politics</b>,
 <b>olive presses</b>,
 <b>good harvest</b>,
 <b>demonstrate</b>,
 <b>could</b>,
 <b>local politics</b>,
 <b>Lydians</b>,
 <b>Medes</b>,
 <b>Persians</b>,
 <b>predicted</b>,
 <b>eclipse</b>,
 <b>negotiations</b>,
 <b>favourable terms</b>,
 <b>died</b>,
 <b>history</b>,
 <b>legend</b>,
 <b>Seven Sages</b>,
 <b>S

In [160]:
print(content[0].string)
print(content2[0].string)

Anaximander
Thales of Miletus


### General Functions

In [161]:
def add_philosopher(url, name, phil_dict, time_period, birth='BC', death='BC', western=True):
    r = get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # Unidecode first paragraph
    par = unidecode(soup.select('p')[0].get_text())
    sentences = par.split('.')
    
    # Calculate lifespan of philosopher
    all=string.maketrans('','')
    nodigs=all.translate(all, string.digits)
    
    # Check for sentence that contains years person lived
    for sentence in sentences:
        lifespan = sentence.translate(all, nodigs)
        if len(lifespan) > 2:
            break
            
    # Check if there were extra numbers in the sentence
    if len(lifespan) > 8:
        lifespan = lifespan[:8]
        
    # Set Philosopher information (Make year negative if BC)
    middle = len(lifespan) / 2
    try:
        phil_dict[name]['year_born'] = -1 * int(lifespan[:middle]) if birth == 'BC' else int(lifespan[:middle])
        phil_dict[name]['year_died'] = -1 * int(lifespan[middle:]) if death == 'BC' else int(lifespan[middle:])
        
    # Fill in with null values if can't be read in
    except ValueError:
        phil_dict[name]['year_born'] = float('NaN')
        phil_dict[name]['year_died'] = float('NaN')
        
    phil_dict[name]['time_period'] = time_period
    phil_dict[name]['Western?'] = western
    
    return phil_dict

In [162]:
def get_image(name, phil_dict):
    img_base = 'http://www.philosophybasics.com/photos/'
    img_url = img_base + name.lower() + '.jpg'
    filepath = 'images/' + name.lower() + '.jpg'
    urllib.urlretrieve(img_url, filepath)
    phil_dict[name]['Image'] = filepath
    
    return phil_dict

### Ancient Time Period

In [163]:
# Obtain Name, birth year, and death year for philosophers of given time period
def ancient_time_period(time_period):
    # Make Request to page of specific time period
    r = get('http://www.philosophybasics.com/historical_' + time_period + '.html')
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # Get names of each philosopher from specified time period
    philosophers = soup.select('a')
    if time_period == 'presocratic':
        min_slice, max_slice = 11, 23
        
    elif time_period == 'socratic':
        min_slice, max_slice = 10, 14
    
    elif time_period == 'hellenistic':
        min_slice, max_slice = 10, 15
    
    else: # time_period == 'roman'
        min_slice, max_slice = 9, 14 
    
    philosophers = [unidecode(x.string) for x in philosophers[min_slice:max_slice]]
    
    # Base url for web pages
    base_url = 'http://www.philosophybasics.com/philosophers_'
    phil_dict = DefaultOrderedDict(dict)
    
    # Determine url to get request from
    if time_period == 'presocratic':
        for name in philosophers:
                
            if name == 'Zeno of Elea':
                name = 'Zeno_Elea'
                
            else:
                # Split name into needed part only
                name = name.split()[0]
                
            
            url = base_url + name.lower() + '.html'
            
            phil_dict = get_image(name, phil_dict)
            
            phil_dict = add_philosopher(url, name, phil_dict, time_period)
                
    elif time_period == 'socratic':
        for name in philosophers:
            name = name.split()[0]
                
            url = base_url + name.lower() + '.html'
            
            phil_dict = get_image(name, phil_dict)
                
            phil_dict = add_philosopher(url, name, phil_dict, time_period)
    
    elif time_period == 'hellenistic':
        for name in philosophers:
            if name == 'Zeno of Citium':
                name = 'Zeno'
                birth, death = 'BC', 'BC'
                url = base_url + 'zeno_citium.html'
                
            elif name == 'Philo of Alexandria':
                name = 'Philo'
                birth, death = 'BC', 'AC'
                url = base_url + name.lower() + '.html'
                
            else:
                birth, death = 'BC', 'BC'
                url = base_url + name.lower() + '.html'
            
            if name == 'Plotinus':
                birth, death = 'AC', 'AC'
                
            phil_dict = get_image(name, phil_dict)
            
            phil_dict = add_philosopher(url, name, phil_dict, time_period, birth=birth, death=death)
            
    else: # time_period == 'roman'
        for name in philosophers:
            if name == 'Cicero, Marcus Tullius':
                name = 'Cicero'
                birth, death = 'BC', 'BC'
                url = base_url + name.lower() + '.html'
            
            elif name == 'St. Augustine of Hippo':
                name = 'Augustine'
                birth, death = 'AD', 'AD'
                url = base_url + name.lower() + '.html'
                
            else:
                name = name.split(r'[, \. ' ']')[0]
                birth, death = 'AD', 'AD'
                url = base_url + name.lower() + '.html'
            
            phil_dict = get_image(name, phil_dict)
            
            phil_dict = add_philosopher(url, name, phil_dict, time_period, birth=birth, death=death)
        
    return phil_dict

#### Pre-Socratic

In [164]:
pre_socratic = ancient_time_period('presocratic')

In [165]:
print(pre_socratic['Parmenides'])
print('\n', pre_socratic.keys())

{'time_period': 'presocratic', 'year_died': -450, 'Image': 'images/parmenides.jpg', 'year_born': -515, 'Western?': True}

['Thales', 'Anaximander', 'Anaximenes', 'Pythagoras', 'Heraclitus', 'Parmenides', 'Anaxagoras', 'Empedocles', 'Zeno_Elea', 'Protagoras', 'Gorgias', 'Democritus']


#### Socratic

In [166]:
socratic = ancient_time_period('socratic')

In [167]:
print(socratic['Socrates'])
print('\n', socratic.keys())

{'time_period': 'socratic', 'year_died': -399, 'Image': 'images/socrates.jpg', 'year_born': -469, 'Western?': True}

['Socrates', 'Plato', 'Diogenes', 'Aristotle']


#### Hellenistic

In [168]:
hellenistic = ancient_time_period('hellenistic')

In [169]:
print(hellenistic['Pyrrho'])
print('\n', hellenistic.keys())

{'time_period': 'hellenistic', 'year_died': -270, 'Image': 'images/pyrrho.jpg', 'year_born': -360, 'Western?': True}

['Pyrrho', 'Epicurus', 'Zeno', 'Philo', 'Plotinus']


#### Roman

In [170]:
roman = ancient_time_period('roman')

In [171]:
print(roman['Epictetus'])
print('\n', roman.keys())

{'time_period': 'roman', 'year_died': 135, 'Image': 'images/epictetus.jpg', 'year_born': 55, 'Western?': True}

['Cicero', 'Epictetus', 'Marcus Aurelius', 'Augustine', 'Boethius']


In [172]:
def ancient_philosophers():
    pre_socratic = ancient_time_period('presocratic')
    socratic = ancient_time_period('socratic')
    hellenistic = ancient_time_period('hellenistic')
    roman = ancient_time_period('roman')
    
    ancient = pre_socratic
    ancient.update(socratic)
    ancient.update(hellenistic)
    ancient.update(roman)
    
    return ancient

In [173]:
ancient = ancient_philosophers()

In [174]:
print(ancient['Aristotle'])
print('\n', ancient.keys())

{'time_period': 'socratic', 'year_died': -322, 'Image': 'images/aristotle.jpg', 'year_born': -384, 'Western?': True}

['Thales', 'Anaximander', 'Anaximenes', 'Pythagoras', 'Heraclitus', 'Parmenides', 'Anaxagoras', 'Empedocles', 'Zeno_Elea', 'Protagoras', 'Gorgias', 'Democritus', 'Socrates', 'Plato', 'Diogenes', 'Aristotle', 'Pyrrho', 'Epicurus', 'Zeno', 'Philo', 'Plotinus', 'Cicero', 'Epictetus', 'Marcus Aurelius', 'Augustine', 'Boethius']


### Medieval Time Period

In [175]:
def medieval_time_period(time_period):
    # Make Request to page of specific time period
    r = get('http://www.philosophybasics.com/historical_' + time_period + '.html')
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # Get names of each philosopher from specified time period
    philosophers = soup.select('a')
    
    if time_period == 'medieval':
        min_slice, max_slice = 8, 18

    else: # time_period == 'renaissance'
        min_slice, max_slice = 12, 16
        
    philosophers = [unidecode(x.string) for x in philosophers[min_slice:max_slice]]
    
    base_url = 'http://www.philosophybasics.com/philosophers_'
    phil_dict = DefaultOrderedDict(dict)
    
    if time_period == 'medieval':
        
        for name in philosophers:
            
            if name == 'Bacon, Roger':
                url = base_url + 'bacon_roger.html'
            
            else:
                name = name.split(r'[, ' ']')[0]
                url = base_url + name.lower() + '.html'
                
            birth, death = 'AD', 'AD'
            
            phil_dict = get_image(name, phil_dict)
            
            phil_dict = add_philosopher(url, name, phil_dict, time_period, birth=birth, death=death)
        
    else: # time_period == 'Renaissance'
        
        for name in philosophers:
            
            if name == 'Bacon, Sir Francis':
                name = 'Sir_Francis_Bacon'
                url = base_url + 'bacon_francis.html'
            
            else:
                name = name.split(',')[0]
                url = base_url + name.lower() + '.html'
            
            birth, death = 'AD', 'AD'
            
            phil_dict = get_image(name, phil_dict)
            
            phil_dict = add_philosopher(url, name, phil_dict, time_period, birth=birth, death=death)
    
    return phil_dict

#### Medieval

In [176]:
medieval_time = medieval_time_period('medieval')

In [177]:
print(medieval_time['Bacon, Roger'])
print('\n', medieval_time.keys())

{'time_period': 'medieval', 'year_died': 1294, 'Image': 'images/bacon, roger.jpg', 'year_born': 1214, 'Western?': True}

['Avicenna (Ibn Sina)', 'Anselm, St.', 'Abelard, Peter', 'Averroes (Ibn Rushd)', 'Maimonides', 'Albertus Magnus', 'Bacon, Roger', 'Aquinas, St. Thomas', 'Scotus, John Duns', 'Ockham (Occam), William of']


#### Renaissance

In [178]:
renaissance = medieval_time_period('renaissance')

In [179]:
print(renaissance['Machiavelli'])
print('\n', renaissance.keys())

{'time_period': 'renaissance', 'year_died': 1527, 'Image': 'images/machiavelli.jpg', 'year_born': 1469, 'Western?': True}

['Erasmus', 'Machiavelli', 'More', 'Sir_Francis_Bacon']


In [180]:
def medieval_philosophers():
    medieval = medieval_time_period('medieval')
    renaissance = medieval_time_period('renaissance')
    
    medieval = medieval
    medieval.update(renaissance)
    
    return medieval

In [181]:
medieval = medieval_philosophers()

In [182]:
print(medieval['Erasmus'])
print('\n', medieval.keys())

{'time_period': 'renaissance', 'year_died': 1536, 'Image': 'images/erasmus.jpg', 'year_born': 1466, 'Western?': True}

['Avicenna (Ibn Sina)', 'Anselm, St.', 'Abelard, Peter', 'Averroes (Ibn Rushd)', 'Maimonides', 'Albertus Magnus', 'Bacon, Roger', 'Aquinas, St. Thomas', 'Scotus, John Duns', 'Ockham (Occam), William of', 'Erasmus', 'Machiavelli', 'More', 'Sir_Francis_Bacon']


### Modern Time Period

In [183]:
def modern_time_period(time_period):
    # Make Request to page of specific time period
    r = get('http://www.philosophybasics.com/historical_' + time_period + '.html')
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # Get names of each philosopher from specified time period
    philosophers = soup.select('a')
    
    if time_period == 'reason' or time_period == 'enlightenment':
        min_slice, max_slice = 9, 16

    else: # time_period == 'modern'
        min_slice, max_slice = 8, 36
        
    philosophers = [unidecode(x.string) for x in philosophers[min_slice:max_slice]]
    
    base_url = 'http://www.philosophybasics.com/philosophers_'
    phil_dict = DefaultOrderedDict(dict)
    
    if time_period == 'reason':
        for name in philosophers:
            
            name_temp = name.split(',')[0]
            
            url = base_url + name_temp.lower() + '.html'
            birth, death = 'AD', 'AD'
            
            phil_dict = get_image(name, phil_dict)
            
            phil_dict = add_philosopher(url, name, phil_dict, time_period, birth=birth, death=death)
    
    elif time_period == 'enlightenment':
        for  name in philosophers:
            
            name_temp = name.split(',')[0]
            
            url = base_url + name_temp.lower() + '.html'
            birth, death = 'AD', 'AD'
            
            phil_dict = get_image(name, phil_dict)
            
            phil_dict = add_philosopher(url, name, phil_dict, time_period, birth=birth, death=death)
            
    else: # time_period == 'modern'
        for name in philosophers:
            
            if name == 'Friedrich Schelling':
                name_temp = 'Schelling'
            
            else:
                name_temp = name.split(',')[0]
            
            url = base_url + name_temp.lower() + '.html'
            birth, death = 'AD', 'AD'
            
            phil_dict = get_image(name, phil_dict)
            
            phil_dict = add_philosopher(url, name, phil_dict, time_period, birth=birth, death=death)
            
    return phil_dict

#### Age of Reason

In [184]:
reason = modern_time_period('reason')

In [185]:
print(reason['Locke, John'])
print('\n', reason.keys())

{'time_period': 'reason', 'year_died': 1704, 'Image': 'images/locke, john.jpg', 'year_born': 1632, 'Western?': True}

['Hobbes, Thomas', 'Descartes, Rene', 'Pascal, Blaise', 'Spinoza, Baruch (Benedict)', 'Locke, John', 'Malebranche, Nicolas', 'Leibniz, Gottfried Wilhelm']


#### Age of Enlightenment

In [186]:
enlightenment = modern_time_period('enlightenment')

In [187]:
print(enlightenment['Smith, Adam'])
print('\n', enlightenment.keys())

{'time_period': 'enlightenment', 'year_died': 1790, 'Image': 'images/smith, adam.jpg', 'year_born': 1723, 'Western?': True}

['Berkeley, Bishop George', 'Voltaire (Francois Marie Arouet)', 'Hume, David', 'Rousseau, Jean-Jacques', 'Smith, Adam', 'Kant, Immanuel', 'Burke, Edmund']


#### Modern

In [188]:
modern_time = modern_time_period('modern')

In [189]:
print(modern_time['Nietzsche, Friedrich'])
print('\n', modern_time.keys())

{'time_period': 'modern', 'year_died': 1900, 'Image': 'images/nietzsche, friedrich.jpg', 'year_born': 1844, 'Western?': True}

['Bentham, Jeremy', 'Fichte, Johann Gottlieb', 'Hegel, G.W.F.', 'Friedrich Schelling', 'Schopenhauer, Arthur', 'Comte, Auguste', 'Emerson, Ralph Waldo', 'Mill, John Stuart', 'Kierkegaard, Soren', 'Thoreau, Henry David', 'Marx, Karl', 'Peirce, Charles Sanders', 'James, William', 'Nietzsche, Friedrich', 'Frege, Gottlob', 'Dewey, John', 'Husserl, Edmund', 'Whitehead, Alfred North', 'Russell, Bertrand', 'Moore, George Edward', 'Wittgenstein, Ludwig', 'Heidegger, Martin', 'Ryle, Gilbert', 'Sartre, Jean-Paul', 'Quine, Willard Van Orman', 'Ayer, Alfred', 'Foucault, Michel', 'Derrida, Jacques']


In [190]:
def modern_philosophers():
    reason = modern_time_period('reason')
    enlightenment = modern_time_period('enlightenment')
    modern = modern_time_period('modern')
    
    modern_phil = reason
    modern_phil.update(enlightenment)
    modern_phil.update(modern)
    
    return modern_phil

In [191]:
modern = modern_philosophers()

In [192]:
print(modern['Kant, Immanuel'])
print('\n', modern.keys())

{'time_period': 'enlightenment', 'year_died': 1804, 'Image': 'images/kant, immanuel.jpg', 'year_born': 1724, 'Western?': True}

['Hobbes, Thomas', 'Descartes, Rene', 'Pascal, Blaise', 'Spinoza, Baruch (Benedict)', 'Locke, John', 'Malebranche, Nicolas', 'Leibniz, Gottfried Wilhelm', 'Berkeley, Bishop George', 'Voltaire (Francois Marie Arouet)', 'Hume, David', 'Rousseau, Jean-Jacques', 'Smith, Adam', 'Kant, Immanuel', 'Burke, Edmund', 'Bentham, Jeremy', 'Fichte, Johann Gottlieb', 'Hegel, G.W.F.', 'Friedrich Schelling', 'Schopenhauer, Arthur', 'Comte, Auguste', 'Emerson, Ralph Waldo', 'Mill, John Stuart', 'Kierkegaard, Soren', 'Thoreau, Henry David', 'Marx, Karl', 'Peirce, Charles Sanders', 'James, William', 'Nietzsche, Friedrich', 'Frege, Gottlob', 'Dewey, John', 'Husserl, Edmund', 'Whitehead, Alfred North', 'Russell, Bertrand', 'Moore, George Edward', 'Wittgenstein, Ludwig', 'Heidegger, Martin', 'Ryle, Gilbert', 'Sartre, Jean-Paul', 'Quine, Willard Van Orman', 'Ayer, Alfred', 'Foucault,

In [193]:
def western_philosophers():
    ancient = ancient_philosophers()
    medieval = medieval_philosophers()
    modern = modern_philosophers()
    
    western = ancient
    western.update(medieval)
    western.update(modern)
    
    return western

In [194]:
western = western_philosophers()

In [195]:
print(western.keys())

['Thales', 'Anaximander', 'Anaximenes', 'Pythagoras', 'Heraclitus', 'Parmenides', 'Anaxagoras', 'Empedocles', 'Zeno_Elea', 'Protagoras', 'Gorgias', 'Democritus', 'Socrates', 'Plato', 'Diogenes', 'Aristotle', 'Pyrrho', 'Epicurus', 'Zeno', 'Philo', 'Plotinus', 'Cicero', 'Epictetus', 'Marcus Aurelius', 'Augustine', 'Boethius', 'Avicenna (Ibn Sina)', 'Anselm, St.', 'Abelard, Peter', 'Averroes (Ibn Rushd)', 'Maimonides', 'Albertus Magnus', 'Bacon, Roger', 'Aquinas, St. Thomas', 'Scotus, John Duns', 'Ockham (Occam), William of', 'Erasmus', 'Machiavelli', 'More', 'Sir_Francis_Bacon', 'Hobbes, Thomas', 'Descartes, Rene', 'Pascal, Blaise', 'Spinoza, Baruch (Benedict)', 'Locke, John', 'Malebranche, Nicolas', 'Leibniz, Gottfried Wilhelm', 'Berkeley, Bishop George', 'Voltaire (Francois Marie Arouet)', 'Hume, David', 'Rousseau, Jean-Jacques', 'Smith, Adam', 'Kant, Immanuel', 'Burke, Edmund', 'Bentham, Jeremy', 'Fichte, Johann Gottlieb', 'Hegel, G.W.F.', 'Friedrich Schelling', 'Schopenhauer, Arthur'

### Find Nationalities and Standardize Names of Philosophers

In [196]:
def find_nationality(d):
    time_periods = ['presocratic', 'socratic', 'hellenistic', 'roman', 'medieval', 'renaissance', 'reason', \
                   'enlightenment', 'modern']
    for time_period in time_periods:
        url = 'http://www.philosophybasics.com/historical_' + time_period + '.html'
        r = get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        content = soup.select('font font')
        
        nationals = []
        for i in range(len(content)):
            lst_words = [unidecode(x) for x in content[i].get_text().strip().split('\n')]
            if len(lst_words) > 1:
                nationals.append(lst_words)
        
        nationals = [item for lst in nationals for item in lst]
        
        i = 0
        for name in d.iterkeys():
            if d[name]['time_period'] == time_period:
                
                components = re.split(r'\)', nationals[i])
            
                d[name]['Nationality'] = components[-1].strip()
                
                i += 1

    return d

In [197]:
find_nationality(western)

DefaultOrderedDict([('Thales',
                     {'Image': 'images/thales.jpg',
                      'Nationality': 'Greek',
                      'Western?': True,
                      'time_period': 'presocratic',
                      'year_born': -624,
                      'year_died': -546}),
                    ('Anaximander',
                     {'Image': 'images/anaximander.jpg',
                      'Nationality': 'Greek',
                      'Western?': True,
                      'time_period': 'presocratic',
                      'year_born': -610,
                      'year_died': -546}),
                    ('Anaximenes',
                     {'Image': 'images/anaximenes.jpg',
                      'Nationality': 'Greek',
                      'Western?': True,
                      'time_period': 'presocratic',
                      'year_born': -585,
                      'year_died': -525}),
                    ('Pythagoras',
                     {'Image': '

In [198]:
def standardize_names(d):
    new_d = DefaultOrderedDict(dict)
    for key, value in d.iteritems():
        components = re.split(r',', key)
        new_key = ' '.join(x for x in components[::-1])
        print(components)
        print(new_key)
        
        old_filepath = 'images/' + key.lower() + '.jpg'
        new_filepath = 'images/' + new_key.strip() + '.jpg'
        
        os.rename(old_filepath, new_filepath)
        new_d[new_key.strip()] = value
    
    return new_d

In [199]:
western = standardize_names(western)

['Thales']
Thales
['Anaximander']
Anaximander
['Anaximenes']
Anaximenes
['Pythagoras']
Pythagoras
['Heraclitus']
Heraclitus
['Parmenides']
Parmenides
['Anaxagoras']
Anaxagoras
['Empedocles']
Empedocles
['Zeno_Elea']
Zeno_Elea
['Protagoras']
Protagoras
['Gorgias']
Gorgias
['Democritus']
Democritus
['Socrates']
Socrates
['Plato']
Plato
['Diogenes']
Diogenes
['Aristotle']
Aristotle
['Pyrrho']
Pyrrho
['Epicurus']
Epicurus
['Zeno']
Zeno
['Philo']
Philo
['Plotinus']
Plotinus
['Cicero']
Cicero
['Epictetus']
Epictetus
['Marcus Aurelius']
Marcus Aurelius
['Augustine']
Augustine
['Boethius']
Boethius
['Avicenna (Ibn Sina)']
Avicenna (Ibn Sina)
['Anselm', ' St.']
 St. Anselm
['Abelard', ' Peter']
 Peter Abelard
['Averroes (Ibn Rushd)']
Averroes (Ibn Rushd)
['Maimonides']
Maimonides
['Albertus Magnus']
Albertus Magnus
['Bacon', ' Roger']
 Roger Bacon
['Aquinas', ' St. Thomas']
 St. Thomas Aquinas
['Scotus', ' John Duns']
 John Duns Scotus
['Ockham (Occam)', ' William of']
 William of Ockham (Occam